<a href="https://colab.research.google.com/github/ccarpenterg/Coursera-Machine-Learning-Stanford/blob/master/03b_NLP_and_recurrent_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torchtext import data

SEED = 3773

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

In [2]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 164k/84.1M [00:00<00:57, 1.46MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 68.2MB/s]


In [3]:
print("Number of training examples: {}".format(len(train_data)))
print("Number of testing examples: {}".format(len(test_data)))

Number of training examples: 25000
Number of testing examples: 25000


In [4]:
print(vars(train_data.examples[0]))

{'text': ['One', 'of', 'the', 'most', 'interesting', 'movies', 'to', 'be', 'classified', 'as', '"', 'blaxploitation', ',', '"', 'Bakshi', "'s", '"', 'Coonskin', '"', 'is', 'a', 'rich', 'text', 'full', 'of', 'wonderful', 'insight', '.', 'He', 'wrote', 'it', 'in', 'collaboration', 'with', 'Scat', 'Man', 'Crothers', 'and', 'Barry', 'White', ',', 'who', 'appear', 'in', 'the', 'film', 'as', 'well', '.', 'The', 'racist', 'imagery', 'can', 'often', 'be', 'disturbing', ',', 'but', 'the', 'message', 'of', 'the', 'movie', 'was', 'so', 'powerful', 'that', 'the', 'NAACP', 'gave', 'it', 'an', 'endorsement', '(', 'but', 'only', 'grudgingly).<br', '/><br', '/>I', 'highly', 'recommend', 'this', 'movie', 'to', 'anyone', 'who', 'is', 'interested', 'in', 'an', 'examination', 'of', 'the', 'pervading', 'atmosphere', 'of', 'racism', 'that', 'Bakshi', 'attempts', 'to', 'deconstruct', '.', 'Wonderful', 'stuff', '.'], 'label': 'pos'}


In [5]:
import random

train_data, valid_data = train_data.split(
    random_state=random.seed(SEED),
    split_ratio=0.8
)

print("Number of training examples: {}".format(len(train_data)))
print("Number of validation examples: {}".format(len(valid_data)))
print("Number of testing examples: {}".format(len(test_data)))

Number of training examples: 20000
Number of validation examples: 5000
Number of testing examples: 25000


In [0]:
MAX_VOCAB_SIZE = 8185

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [7]:
print("Unique tokens in TEXT vocabulary: {}".format(len(TEXT.vocab)))
print("Unique tokens in LABEL vocabulary: {}".format(len(LABEL.vocab)))

Unique tokens in TEXT vocabulary: 8187
Unique tokens in LABEL vocabulary: 2


In [8]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 232172), (',', 220766), ('.', 188996), ('a', 125361), ('and', 125203), ('of', 115386), ('to', 107089), ('is', 87673), ('in', 70335), ('I', 61669), ('it', 61442), ('that', 56596), ('"', 50771), ("'s", 49648), ('this', 48163), ('-', 42485), ('/><br', 40881), ('was', 40134), ('as', 34899), ('with', 34388)]


In [9]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


In [10]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f82043de378>, {'pos': 0, 'neg': 1})


In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device
)